In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.pyfunc
from mlflow.models import infer_signature

2025-12-11 17:03:42.078652: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/siddhu/Desktop/Machine-Learning-Operations/venv/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# load the data
data = pd.read_csv('https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv', sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
## Split the data to training, validation and test test
train, test = train_test_split(data, test_size=0.2, random_state=42)

train_x = train.drop(columns=['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

# test data
test_x = test.drop(columns=['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

#train and valid data
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

signature = infer_signature(train_x, train_y)

In [4]:
## ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    ## define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params['lr'], momentum=params['momentum']), loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError()])

    ## Train the ANN model with lr and momentum params with MLFLOW params
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=epochs, batch_size=64)
        ## Evaluate the model
        eval_res = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_res[1]
        ## log params and results
        mlflow.log_param('params', params)
        mlflow.log_metric("eval_rmse", eval_rmse)
        ## log the model
        mlflow.tensorflow.log_model(model=model, name='dl_model', signature=signature)

        return {'loss': eval_rmse, 'status': STATUS_OK, 'model': model} 

In [5]:
def objective(params):
    # MLFlow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )

    return result

In [6]:
space = {
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-1)),
    'momentum': hp.uniform('momentum', 0.0, 1.0)
}

In [7]:
mlflow.set_experiment('/wine-quality')
with mlflow.start_run():
    # conduct the hyperparameter tuning search using hyperopt
    trials = Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=5,
        trials=trials
    )

    # fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]

    # log the best parameters, loss and model
    mlflow.log_params(best)

    mlflow.log_metric('eval_rmse', best_run['loss'])
    mlflow.tensorflow.log_model(model=best_run['model'], name='model', signature=signature)

    # print out the best parameters and corresponding loss
    print(f'Best parameters : {best}')
    print(f'Best eval rmse : {best_run['loss']}')

2025/12/11 17:03:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/11 17:03:46 INFO mlflow.store.db.utils: Updating database tables
2025/12/11 17:03:46 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/11 17:03:46 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/11 17:03:46 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/11 17:03:46 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/11 17:03:46 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

W0000 00:00:1765451926.855278   21609 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 16s 340ms/step - loss: 34.3291 - root_mean_squared_error: 5.8591
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.3722 - root_mean_squared_error: 2.8935 - val_loss: 2.1535 - val_root_mean_squared_error: 1.4675

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.3209 - root_mean_squared_error: 1.1493
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4222 - root_mean_squared_error: 1.1926 - val_loss: 1.4984 - val_root_mean_squared_error: 1.2241

Epoch 3/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.3172 - root_mean_squared_error: 1.1477
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1114 - root_mean_squared_error: 1.0542 - val_loss: 1.1992 - val_root_mean_squared_error: 1.0951

 1/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.1078 - root_mean_squared_error: 1.0525
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [9]:
# Load the active model
model = mlflow.pyfunc.load_model("models:/m-b6f3b75df6984a958b0165048b08d121")  # or Staging
# Use it with X_test
predictions = model.predict(test_x)
predictions

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.9626126],
       [7.17333  ],
       [6.582364 ],
       [5.8885846],
       [6.6440215],
       [6.3476014],
       [4.9974008],
       [5.192568 ],
       [6.16004  ],
       [5.179303 ],
       [6.450623 ],
       [5.1202626],
       [6.9143353],
       [5.7039685],
       [6.0363016],
       [5.5461917],
       [6.7226505],
       [5.9376173],
       [6.210736 ],
       [5.6368423],
       [5.595269 ],
       [6.1671357],
       [5.5889583],
       [6.0365   ],
       [6.0633144],
       [5.699507 ],
       [5.675709 ],
       [6.4865327],
       [6.135762 ],
       [5.602266 ],
       [5.475212 ],
       [5.73842  ],
       [5.6230936],
       [5.5599484],
       [5.747871 ],
       [6.786101 ],
       [6.317978 ],
       [5.329967 ],
       [5.6397867],
       [5.9211326],
       [5.5253773],
       [5.580015 ],
       [6.0149465],
       [5.902563 ],
       [5.400066 ],
       [5.7563205],
       [5.6848674],
       [5.259577 ],
       [5.649048 ],
       [5.578134 ],
